# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)

Seu código deve ser testado na seguinte instância: [link](https://drive.google.com/file/d/12CeZEow-6vVgJFgzXMo0gbjV5hLCM6Zi/view?usp=sharing). O README se encontra em: [link](https://drive.google.com/file/d/1LBTdkDoTQRxUJsKLI4Z38-Uc8oUPCZQ0/view?usp=sharing).

In [26]:
import time, heapq

class Item:
  def __init__(self, id, valor, peso):
    self.id = id
    self.valor = valor
    self.peso = peso
    self.fracionario = valor / peso

class Noh:
  def __init__(self, nivel, valor, peso, limitante, itens):
    self.nivel = nivel
    self.valor = valor
    self.peso = peso
    self.limitante = limitante
    self.itens = itens

  # Problema de Maximização: a min-heap elimina primeiro os nós de maior limitante superior (max-heap)
  def __lt__(self, other):
    return self.limitante > other.limitante

# Heurística gulosa para determinar valor máximo obtido na mochila fracionária
def guloso(itens, capacidade_mochila):
  itens = sorted(itens, key=lambda x: x.fracionario, reverse=True)
  valor_total = 0

  for item in itens:
    if capacidade_mochila - item.peso >= 0:
      capacidade_mochila -= item.peso
      valor_total += item.valor
    else:
      valor_total += item.fracionario * capacidade_mochila
      break
  return valor_total

# Calcula o limitante superior do nó (usando mochila fracionária)
def calcular_limitante(noh, capacidade, itens):
    if noh.peso >= capacidade:
        return 0  # Inviável

    limitante = noh.valor
    peso_total = noh.peso
    nivel = noh.nivel

    # Adiciona itens para calcular o limitante
    for i in range(nivel, len(itens)):
        if peso_total + itens[i].peso <= capacidade:
            peso_total += itens[i].peso
            limitante += itens[i].valor
        else:
            limitante += itens[i].valor * (capacidade - peso_total) / itens[i].peso
            break
    return limitante

# A árvore de exploração do Branch&Bound é binária -> {incluir o item, não incluir o item}
def branch_and_bound(itens, capacidade_mochila):
  inicio = time.time()
  itens = sorted(itens, key=lambda x: x.fracionario, reverse=True) # Ordena os itens por custo benefício em ordem decrescente

  melhor_valor = 0
  nos_explorados = 0
  melhor_solucao = []

  fila = []
  raiz = Noh(nivel=0, valor=0, peso=0, limitante= guloso(itens, capacidade_mochila), itens=[])
  heapq.heappush(fila, raiz)

  while fila:
    noh_atual = heapq.heappop(fila)
    nos_explorados += 1

    if noh_atual.limitante > melhor_valor: # Se o nó extraído é promissor
      nivel = noh_atual.nivel
      if nivel < len(itens): # Se o nível da árvore não ultrapassa a quantidade de itens disponíveis

        # Cria um nó para a situação de incluir o item
        noh_incluir = Noh(
            nivel = nivel+1,
            valor = noh_atual.valor + itens[nivel].valor,
            peso = noh_atual.peso + itens[nivel].peso,
            limitante=0,
            itens = noh_atual.itens + [itens[nivel].id])

        # Atualiza a solução ótima do problema
        if noh_incluir.peso <= capacidade_mochila and noh_incluir.valor > melhor_valor:
              melhor_valor = noh_incluir.valor
              melhor_solucao = noh_incluir.itens

        # Calcula novo limitante superior, com a inclusão do item
        noh_incluir.limitante = calcular_limitante(noh_incluir, capacidade_mochila, itens)

        # Se esse novo nó (incluindo o item) for promissor, inclui-o na fila para exploração
        if noh_incluir.limitante > melhor_valor:
            heapq.heappush(fila, noh_incluir)

        # Cria um nó para a situação de não incluir o próximo item
        noh_nao_incluir = Noh(
            nivel=nivel + 1,
            valor=noh_atual.valor,
            peso=noh_atual.peso,
            limitante= calcular_limitante(noh_atual, capacidade_mochila, itens),
            itens=noh_atual.itens
        )

        # Se esse novo nó (não incluindo o item) for promissor, inclui-o na fila para exploração
        if noh_nao_incluir.limitante > melhor_valor:
            heapq.heappush(fila, noh_nao_incluir)

  fim = time.time()
  gap = (guloso(itens, capacidade_mochila) - melhor_valor) / guloso(itens, capacidade_mochila) * 100
  return melhor_valor, melhor_solucao, nos_explorados, gap, fim - inicio


with open("Weakly001", 'r') as arq:
  instance_name = arq.readline()
  instance_class = arq.readline()
  capacidade_mochila = int(arq.readline())
  itens = []
  i = 0
  for line in arq:
    valor, peso = line.split()
    itens.append(Item(i, int(valor), int(peso)))
    i += 1

melhor_valor, melhor_solucao, nos_explorados, gap, tempo_gasto = branch_and_bound(itens, capacidade_mochila)

print(f"Melhor Valor: {melhor_valor}")
print(f"Melhor Solução (itens): {melhor_solucao}")
print(f"Nós Explorados: {nos_explorados}")
print(f"GAP de Otimalidade: {gap:.2f}%")
print(f"Tempo de execução gasto: {tempo_gasto:.4f} segundos")

Melhor Valor: 4323
Melhor Solução (itens): [676, 348, 436, 381, 311, 645, 786, 432, 140, 352, 462, 190, 454, 439, 973, 562, 480, 915, 320, 823, 520, 428, 712, 238, 738, 385, 567, 904, 466, 456, 749, 175, 908, 430, 233, 378, 148, 473]
Nós Explorados: 1141
GAP de Otimalidade: 0.14%
Tempo de execução gasto: 0.0145 segundos
